# Рекомендация тарифов

Есть данные о поведении клиентов, которые уже перешли на определенные тарифы, и нужно построить модель для задачи классификации, которая сможет выбрать наилучший тариф для новых клиентов. Предобработка данных уже была проведена, так что можно сразу начинать работу с моделью.

Для выполнения данной задачи необходимо построить модель с максимально возможным значением accuracy.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [ ]:
try:
    data = pd.read_csv('/Users/arina200212yandex.ru/Desktop/users_behavior.csv')
except:
    data = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
data.head(15)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [ ]:
data.duplicated().sum()

0

**Вывод**

Предварительная обработка данных не требуется, поэтому можно продолжать работу с данными в исходном виде.

## Разбейте данные на выборки

Для продолжения работы необходимо разделить данные на следующие выборки:

- Обучающая выборка;
- Валидационная выборка;
- Тестовая выборка.

Для этого будут использоваться следующие переменные:

- features_obuchenie, target_obuchenie - для хранения данных обучающей выборки;
- features_valida, target_valida - для хранения данных валидационной выборки;
- features_test, target_test - для хранения данных тестовой выборки.

In [ ]:
obuchenie, test = train_test_split(data, test_size=0.4, random_state=12345)
features = test.drop('is_ultra', axis=1)
target = test['is_ultra']

In [ ]:
obuchenie.shape

(1928, 5)

In [ ]:
features_valida, features_test, target_valida, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)
features_obuchenie = obuchenie.drop('is_ultra', axis=1)
target_obuchenie = obuchenie['is_ultra']

In [ ]:
features_test.shape

(643, 4)

In [ ]:
features_valida.shape

(643, 4)

## Исследуйте модели

Рассмотрим модель "Дерево решений"

In [ ]:
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_obuchenie , target_obuchenie)
    accuracy = model.score(features_valida, target_valida)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "Качество:", best_accuracy)

Глубина дерева: 9 Качество: 0.7869362363919129


Рассмотрим модель "Случаный лес"

In [ ]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_obuchenie , target_obuchenie)
            accuracy = model.score(features_valida, target_valida)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"Качество:", best_accuracy)

Глубина дерева: 7 Количество деревьев: 35 Качество: 0.8009331259720062


Рассмотрим модель "Логическая регрессия"

In [ ]:
model = LogisticRegression()
model.fit(features_obuchenie , target_obuchenie)
accuracy = model.score(features_valida, target_valida)
print("Качество:", accuracy)

Качество: 0.7107309486780715


**Вывод**
Следующие метрики качества демонстрируют, что результаты были достигнуты:

- Качество модели "Дерево решений" (при гиперпараметре max_depth=9): 0.786;
- Качество модели "Случайный лес" (при гиперпараметрах max_depth=7, max_samples_leaf=3 и n_estimators=35): 0.800;
- Качество модели "Логистическая регрессия": 0.710.

Исходя из этих метрик можно сделать вывод, что модель "Случайный лес" проявила себя лучше всего.


## Проверьте модель на тестовой выборке

Поскольку модель "Случайный лес" дала лучшие результаты, мы будем проверять ее производительность на тестовой выборке.

In [ ]:
model = RandomForestClassifier(max_depth=7, n_estimators=35, min_samples_leaf=3, random_state=1234)
model.fit(features_obuchenie , target_obuchenie)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7900466562986003


**Вывод**

Был достигнут хороший результат на тестовой выборке, где значение accuracy составило 0.79. Это было достигнуто с помощью модели Случайного леса, которая была обучена с использованием следующих гиперпараметров: max_depth=7, max_samples_leaf=45 и n_estimators=45.


**ФИНАЛЬНЫЙ ВЫВОД**

Проведено тестирование трех моделей: "Деревья решений", "Случайный лес" и "Логистическая регрессия". Результаты показали, что наилучшее качество достигнуто при использовании модели "Случайный лес" с параметрами max_depth=7, max_samples_leaf=45 и n_estimators=45.